# Lee Carter

## Packages

The demography package allows you to access data from directly from the [Human Mortality Database](https://www.mortality.org/). The StMoMo package allows one to fit a variety of mortality models.

In [11]:
library(StMoMo)
library(demography)
library(rjson)

## Data

We import the data for _England and Whales_. Since we're aiming to fit the Poisson and Negative Binomial versions of the Lee-Carter model, we'll be using the central mortality data.

In [15]:
cred <- fromJSON(file="../hmd_credentials.json")
username <- cred$username
password <- cred$password

EWdata <- hmd.mx(country = "GBRTENW", username = username, 
                 password = password, label = "GBRTENW")

# central exposure data
EWMaleCenData <- StMoMoData(EWdata, series = "male")
EWFemaleCenData <- StMoMoData(EWdata, series = "female")

## Model

In [6]:
ages <- 19:103
years <- 1950:2018

### Poisson Model

We fit the Poisson Lee-Carter model to both the male and female data.

In [ ]:
LCpoisMale <- fit(lc(link = "log"), data = EWMaleCenData, ages.fit = ages, years.fit = years)
LCpoisFemale <- fit(lc(link = "log"), data = EWFemaleCenData, ages.fit = ages, years.fit = years)

In [ ]:
BIC(LCpoisMale)
BIC(LCpoisFemale)

LCpoisMale$deviance
LCpoisFemale$deviance

### Negative Binomial

LCnegbinom_algorithm.R contains the code to fit the negative binomial version of the Lee-Carter model. The code is an implementation of the algorithm described in _Uncertainty in Mortality Forecasting: An Extension to the Classical Lee-Carter Approach_ in Li2009.

In [10]:
# functions to fit negative binomial version of Lee-Carter model
source(paste0(getwd(), "/LCnegbinom_algorithm.R"))

#### Male

We gather some of the StMoMo output for convenience. We also set up a list of the Poisson maximum likelihood estimates to use as the initial parameters for the Negative Binomial algorithm iteration.

In [ ]:
# death-, exposure- and missing data weight matrix
dxt <- LCpoisMale$data$Dxt
dxt <- dxt[as.integer(rownames(dxt))%in%ages, as.integer(colnames(dxt))%in%years]
Ext <- LCpoisMale$data$Ext
Ext <- Ext[as.integer(rownames(Ext))%in%ages, as.integer(colnames(Ext))%in%years]
wxt <- LCpoisMale$wxt

# fixing layout for Julia read-in
ax <- c(LCpoisMale$ax)
bx <- c(LCpoisMale$bx)
kt <- c(LCpoisMale$kt)
names(ax) <- ages
names(bx) <- ages
names(kt) <- years
# Poisson maximum likelihood estimates of parameters
LCpoisMale <- list(ax=ax, bx=bx, kt=kt)

We use the Poisson maximum likelihood parameters as the initial parameters for the negative binomial algorithm for faster convergence.

Note that the dispersion parameters are estimated using the method of moments
$$E(\lambda_x)=Var\left(\frac{D_{xt}}{\hat{D}_{xt}}\right)$$

We found that the dispersion parameters, especially at the youngest and oldest ages, take some time to converge, while the base Lee-Carter parameters converge reasonably quickly. Hence, we run the model for 2 iterations so that the base Lee-Carter parameters can converge reasonably at most ages and the dispersion parameters can be reestimated according to the new parameters. We then start iteration again using the new initial parameters.

In [ ]:
# using Poisson maximum likelihood parameters as starting parameters for iteration
LCnbMale_init <- LCnegbinom(LCpoisMale, dxt, Ext, wxt, maxiter=2)
LCnbMale <- LCnegbinom(LCnbMale_init, dxt, Ext, wxt, maxiter=1000)

#### Female

We do the same for fitting the female data.

In [ ]:
# death-, exposure- and missing data weight matrix
dxt <- LCpoisFemale$data$Dxt
dxt <- dxt[as.integer(rownames(dxt))%in%ages, as.integer(colnames(dxt))%in%years]
Ext <- LCpoisFemale$data$Ext
Ext <- Ext[as.integer(rownames(Ext))%in%ages, as.integer(colnames(Ext))%in%years]
wxt <- LCpoisFemale$wxt

# fixing layout for Julia read-in
ax <- c(LCpoisFemale$ax)
bx <- c(LCpoisFemale$bx)
kt <- c(LCpoisFemale$kt)
names(ax) <- ages
names(bx) <- ages
names(kt) <- years
# Poisson maximum likelihood estimates of parameters
LCpoisFemale <- list(ax=ax, bx=bx, kt=kt)

In [ ]:
# using Poisson maximum likelihood parameters as starting parameters for iteration
LCnbFemale_init <- LCnegbinom(LCpoisFemale, dxt, Ext, wxt, maxiter=2)
LCnbFemale <- LCnegbinom(LCnbFemale_init, dxt, Ext, wxt, maxiter=1000)

In [ ]:
LCnbFemale$deviance

We fit random walks with drift to $\kappa_t$ of each model. We note that the Negative Binomial Lee-Carter models will have broader forecast intervals (as expected) due to the larger standard deviations of the random walk models.

In [ ]:
# Fitting Random Walks with drift
arima(diff(as.vector(LCpoisMale$kt)), order = c(0, 0, 0), method = "ML")
arima(diff(as.vector(LCpoisFemale$kt)), order = c(0, 0, 0), method = "ML")
arima(diff(LCnbMale$kt), order = c(0, 0, 0), method = "ML")
arima(diff(LCnbFemale$kt), order = c(0, 0, 0), method = "ML")

In [ ]:
# Fitting Random Walks with drift
LCpoisMale_arima <- arima(diff(as.vector(LCpoisMale$kt)), order = c(0, 0, 0), method = "ML")
LCpoisFemale_arima <- arima(diff(as.vector(LCpoisFemale$kt)), order = c(0, 0, 0), method = "ML")
LCnbMale_arima <- arima(diff(LCnbMale$kt), order = c(0, 0, 0), method = "ML")
LCnbFemale_arima <- arima(diff(LCnbFemale$kt), order = c(0, 0, 0), method = "ML")

## Save Model

In [ ]:
collect_arima_info <- function(y, model, manual_diff_order=0){
  order <- c(model$call$order[[2]], model$call$order[[3]], model$call$order[[4]])
  order[2] <- order[2] + manual_diff_order
  collected <- list(order=order, y=y, e=append(rep(0, manual_diff_order), as.vector(model$residuals)))
  if (order[1] >= 1){
    collected[["phi"]] <- as.vector(model$coef[1:order[1]])
  } else{
    collected[["phi"]] <- 0
  }
  
  if (order[3] >= 1){
    collected[["theta"]] <- as.vector(model$coef[(order[1]+1):(order[1]+order[3])])
  } else{
    collected[["theta"]] <- 0
  }
  
  collected[["mu"]] <- ifelse(any("intercept" %in% names(model$coef)), model$coef[["intercept"]], 0)
  collected[["sigma"]] <- model$sigma
  return(collected)
}

In [ ]:
LCpoisMale[["kt"]] <- collect_arima_info(y=LCpoisMale$kt, model=LCpoisMale_arima, manual_diff_order=1)
LCpoisFemale[["kt"]] <- collect_arima_info(y=LCpoisFemale$kt, model=LCpoisFemale_arima, manual_diff_order=1)
LCnbMale[["kt"]] <- collect_arima_info(y=LCnbMale$kt, model=LCnbMale_arima, manual_diff_order=1)
LCnbFemale[["kt"]] <- collect_arima_info(y=LCnbFemale$kt, model=LCnbFemale_arima, manual_diff_order=1)

In [ ]:
save(LCpoisMale, LCpoisFemale, LCnbMale, LCnbFemale, file = "LeeCarter.RData", compress = "gzip")